In [2]:
from configparser import ConfigParser
import psycopg2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import recall_score, precision_score
#Can be very helpful to notice any imbalance in classes
from collections import Counter 

In this part, we try to connect to PSQL

In [3]:
#Sourced from https://www.postgresqltutorial.com/postgresql-python/connect/
def config(filename='database.ini', section='postgresql'):
    parser = ConfigParser()
    # read config file
    parser.read(filename) 
 
    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))
 
    return db

In [4]:
#Get the configuration file as a python dictionary
cfg = config()

In [5]:
#Establish the connection and create a cursor to the database
try:
    print("Here's an attempt to connect to the database")
    conn = psycopg2.connect(**cfg)
    cursor = conn.cursor()
    print("Look's like it was a success")
    
except (Exception, psycopg2.DatabaseError) as error:
    print(error)

Here's an attempt to connect to the database
Look's like it was a success


In [8]:
try:
    #Lets get our data 
    cursor.execute("SELECT * from weather_dimension as weather")

    #Get the complete result set. It will be a list of tuples where each tuple is a row from the result set
    result_list = cursor.fetchall()
        
except (Exception, psycopg2.DatabaseError) as error:
    print(error)

In [9]:
#Ensure to run this cell at the end of all your experiments to close all connections
cursor.close()
conn.close()

Let's prepare the DataFrame.

In [10]:
#Now, 
result_df = pd.DataFrame(result_list)

Let's see how the data looks like

In [11]:
result_df.head()

,0,1,2,3,4
0,0,OTTAWA CDA,32.5,17.5,0.0
1,1,OTTAWA CDA,32.0,21.0,0.8
2,2,OTTAWA CDA,32.0,20.5,0.0
3,3,OTTAWA CDA,30.5,17.5,0.6
4,4,OTTAWA CDA,30.0,15.5,0.0


Let's count the number of instances for each class

Both labels are highly imbalanced! We need to make sure to consider this when we test our models.

We are done with pre-processing of our data. Next, we need to devide our data to training and testing sets.